Original Code. Breakdown into steps, redo the queries as they dont work currently.

In [6]:
import wrds
import polars as pl

def connect_to_wrds():
    # Connect to WRDS using your credentials
    conn = wrds.Connection()
    return conn

In [20]:
# Converter from ticker to permno and gvkey

# Example query to get ticker data, gvkey, and permno from Compustat
def get_gvkey_permno(conn, tickers, start_date, end_date):
    # Connect to WRDS
    
    query = """
    SELECT tic, gvkey, permno
    FROM comp.security
    WHERE tic IS NOT NULL AND gvkey IS NOT NULL AND permno IS NOT NULL
    """
    
    data = conn.raw_sql(query)
    return data

# Example usage
conn = connect_to_wrds()
tickers = ['AAPL']
get_gvkey_permno(conn, tickers, '2023-01-01', '2023-09-01')

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "permno" does not exist
LINE 2:     SELECT tic, gvkey, permno
                               ^

[SQL: 
    SELECT tic, gvkey, permno
    FROM comp.security
    WHERE tic IS NOT NULL AND gvkey IS NOT NULL AND permno IS NOT NULL
    ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [21]:
def get_ohlcv_data(conn, tickers, start_date, end_date):
    # Query CRSP for OHLCV data (daily)
    query = f"""
        SELECT date, security.tic, permno, cusip, prc AS close, 
               bidlo AS low, askhi AS high, vol AS volume, openprc AS open
        FROM crsp.dsf
        WHERE security.tic IN ({','.join([f"'{ticker}'" for ticker in tickers])})
          AND date BETWEEN '{start_date}' AND '{end_date}'
        ORDER BY date
    """
    
    data = conn.raw_sql(query)
    return data

def convert_to_polars(data):
    # Convert to Polars DataFrame
    df = pl.DataFrame(data)
    
    # Ensure the date is in the correct format
    df = df.with_columns(df['date'].str.strptime(pl.Date, format='%Y-%m-%d'))
    
    return df


In [22]:
def convert_to_lean_format_polars(df, ticker, frequency='daily'):
    # Ensure correct order of columns
    df = df.select(['date', 'open', 'high', 'low', 'close', 'volume'])
    
    # Convert date to the required format (YYYYMMDD)
    df = df.with_columns(df['date'].dt.strftime('%Y%m%d').alias('date'))
    
    # Save to the appropriate location
    if frequency == 'daily':
        output_file = f"data/equity/usa/daily/{ticker}.csv"
    elif frequency == 'hourly':
        output_file = f"data/equity/usa/hourly/{ticker}.csv"
    else:
        output_file = f"data/equity/usa/minute/{ticker}.zip"

    # Save as CSV in the required format
    df.write_csv(output_file)


In [23]:
def wrds_pipeline(tickers, start_date, end_date, frequency='daily'):
    # Connect to WRDS
    conn = connect_to_wrds()
    
    for ticker in tickers:
        # Get OHLCV data
        raw_data = get_ohlcv_data(conn, [ticker], start_date, end_date)
        
        # Convert to Polars DataFrame
        df_polars = convert_to_polars(raw_data)
        
        # Convert and save in LEAN format
        convert_to_lean_format_polars(df_polars, ticker, frequency)
    
    # Close the connection when done
    conn.close()

# Example usage
tickers = ['AAPL']
wrds_pipeline(tickers, '2023-01-01', '2023-09-01', 'daily')


OperationalError: (psycopg2.OperationalError) connection to server at "wrds-pgdata.wharton.upenn.edu" (165.123.60.118), port 9737 failed: FATAL:  too many connections for role "mpalanca"

(Background on this error at: https://sqlalche.me/e/20/e3q8)